In [7]:
import meep as mp
import math
import cmath
import argparse



# default unit length is 1 um
um_scale = 1.0


   
    
resolution = 40     # pixels/um

a = 3.4       # lattice periodicity
d = 1.4        # cube side length
#h = 0.2             # metal rod height

tcell= 1.4     # unitcell thickness    
tsub = 4.0          # substrate thickness
tpml = 5.0          # PML thickness
tair = 4.0          # air thickness

sz = tabs+tair+h+tmet+tsub+tabs
cell_size = mp.Vector3(a,a,sz)

pml_layers = [mp.PML(thickness=tabs,direction=mp.Z,side=mp.High),
                  mp.Absorber(thickness=tabs,direction=mp.Z,side=mp.Low)]

lmin = 2.0          # source min wavelength
lmax = 5.0          # source max wavelength
fmin = 1/lmax       # source min frequency
fmax = 1/lmin       # source max frequency
fcen = 0.5*(fmin+fmax)
df = fmax-fmin

nSi = 3.5
Si = mp.Medium(index=nSi)

if args.empty:
    geometry = []
else:
    geometry = [ mp.Cylinder(material=Pt, radius=r, height=h, center=mp.Vector3(0,0,0.5*sz-tabs-tair-0.5*h)),
                     mp.Block(material=Pt, size=mp.Vector3(mp.inf,mp.inf,tmet),
                              center=mp.Vector3(0,0,0.5*sz-tabs-tair-h-0.5*tmet)),
                     mp.Block(material=Si, size=mp.Vector3(mp.inf,mp.inf,tsub+tabs),
                              center=mp.Vector3(0,0,0.5*sz-tabs-tair-h-tmet-0.5*(tsub+tabs))) ]

# CCW rotation angle (degrees) about Y-axis of PW current source; 0 degrees along -z axis
theta = math.radians(args.theta)

# k with correct length (plane of incidence: XZ) 
k = mp.Vector3(math.sin(theta),0,math.cos(theta)).scale(fcen)

def pw_amp(k, x0):
    def _pw_amp(x):
        return cmath.exp(1j * 2 * math.pi * k.dot(x + x0))
    return _pw_amp

src_pos = 0.5*sz-tabs-0.2*tair
sources = [ mp.Source(mp.GaussianSource(fcen, fwidth=df), component=mp.Ey, center=mp.Vector3(0,0,src_pos),
                          size=mp.Vector3(a,a,0),
                          amp_func=pw_amp(k, mp.Vector3(0,0,src_pos))) ]

sim = mp.Simulation(cell_size=cell_size,
                        geometry=geometry,
                        sources=sources,
                        boundary_layers=pml_layers,
                        k_point = k,
                        resolution=resolution)

nfreq = 50
refl = sim.add_flux(fcen, df, nfreq, mp.FluxRegion(center=mp.Vector3(0,0,+0.5*sz-tabs-0.5*tair),size=mp.Vector3(a,a,0)))

if not args.empty:
    sim.load_minus_flux('refl-flux', refl)
    
sim.run(until_after_sources=mp.stop_when_fields_decayed(25, mp.Ey, mp.Vector3(0,0,0.5*sz-tabs-0.5*tair), 1e-7))

if args.empty:
    sim.save_flux('refl-flux', refl)
    
sim.display_fluxes(refl)



NameError: name 'Pt_susc' is not defined